In [1]:
import re
import random
import numpy as np
import os.path
import scipy.misc
import shutil
import zipfile
import time
import tensorflow as tf
from glob import glob
#from urllib.request import urlretrieve
from tqdm import tqdm

import helper
#import project_tests as tests

In [2]:
import sys
sys.version_info

sys.version_info(major=2, minor=7, micro=6, releaselevel='final', serial=0)

In [3]:
tf.saved_model.loader.load
tf.__version__

'1.0.1'

In [4]:
def load_vgg(sess, vgg_path):
    
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    buff = tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    
    image_input = tf.get_default_graph().get_tensor_by_name(vgg_input_tensor_name)
    print ('=== image_input ', image_input.shape)
    print ('=== image_input ', image_input.get_shape)
    
    keep_prob = tf.get_default_graph().get_tensor_by_name(vgg_keep_prob_tensor_name)
    print ('=== keep_prob ', keep_prob.get_shape)
    
    layer3_out = tf.get_default_graph().get_tensor_by_name(vgg_layer3_out_tensor_name)
    print ('=== layer3_out ', layer3_out.get_shape)
    
    layer4_out = tf.get_default_graph().get_tensor_by_name(vgg_layer4_out_tensor_name)
    print ('=== layer4_out ', layer4_out.get_shape)
    
    layer7_out = tf.get_default_graph().get_tensor_by_name(vgg_layer7_out_tensor_name)
    print ('=== layer7_out ', layer7_out.get_shape)
    
    
    
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out


In [5]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    
    # 1x1 with 7
    layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='SAME', 
                              kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))
    #print ('Layers layer7 ' , layer7.shape)
    print ('Layers layer7 ' , layer7.get_shape)
    
    # 1x1 with 4
    layer4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='SAME',
                             kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))
    
    #print ('Layers layer4 ' , layer4.shape) 
    print ('Layers layer4 ' , layer4.get_shape)
    
    # 1x1 with 3
    layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='SAME',
                             kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))
    
    #print ('Layers layer3 ' , layer3.shape) 
    print ('Layers layer3 ' , layer3.get_shape)
    
    # Upsample 7 -- Transposed Convolution on layer7
    upsample7 = tf.layers.conv2d_transpose(layer7, num_classes, 4, 2, 'SAME',
                                          kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))
    
    #print ('Layers upsample7 ' , upsample7.shape) 
    print ('Layers upsample7 ' , upsample7.get_shape)
    
    # Skip-connection with 4 and upsample7
    comb_layer1 = tf.add(layer4, upsample7)
    #print ('Layers comb_layer1 ' , comb_layer1.shape) 
    print ('Layers comb_layer1 ' , comb_layer1.get_shape)
    
    # Upsample combined layer 4+7  -- Transposed Convolution on comb_layer1
    upsample4_7 = tf.layers.conv2d_transpose(comb_layer1, num_classes, 4, 2, 'SAME',
                                            kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))
    
    #print ('Layers upsample4_7 ' , upsample4_7.shape)
    print ('Layers upsample4_7 ' , upsample4_7.get_shape)
    
    # Skip-connection with 3 and upsample4_7
    comb_layer2 = tf.add(layer3, upsample4_7)
    #print ('Layers comb_layer2 ' , comb_layer2.shape)
    print ('Layers comb_layer2 ' , comb_layer2.get_shape)
    
    # Upsample to original image  -- Transposed Convolution on comb_layer2
    upsample3_4_7 = tf.layers.conv2d_transpose(comb_layer2, num_classes, 16, 8, 'SAME',
                                              kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))
    
    #print ('Layers upsample3_4_7 ' , upsample3_4_7.shape) 
    print ('Layers upsample3_4_7 ' , upsample3_4_7.get_shape)
        
    return upsample3_4_7

#tests.test_layers(layers)

In [6]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    reshaped_logits = tf.reshape(nn_last_layer, (-1, num_classes))
    print ('Layers reshaped_logits ' , reshaped_logits.get_shape)
    
    reshaped_labels = tf.reshape(correct_label, (-1, num_classes))
    print ('Layers correct_label  ' , correct_label.get_shape)
    print ('Layers reshaped_labels ' , reshaped_labels.get_shape)
    
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = reshaped_logits, 
                                                                                labels = reshaped_labels))
    
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    
    return reshaped_logits, train_op, cross_entropy_loss
    
#tests.test_optimize(optimize)

In [7]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    
    idx = 0
    for epoch in range(epochs):
        for image, gt_image in get_batches_fn(batch_size):
            idx += 1
            _, loss = sess.run([train_op, cross_entropy_loss], 
                                     feed_dict = {input_image: image, correct_label: gt_image, 
                                                  keep_prob: 0.5, learning_rate: 0.001})
            #print(idx)
            if idx % 8 == 0: 
                print("Epoch {}/{}...".format(epoch, epochs),
                      "Training Loss: {:.4f}...".format(loss))

#tests.test_train_nn(train_nn)

In [8]:
def run():
    
    
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    #tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    
    #Already downloaded
    #helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
    epochs = 50 
    batch_size = 16

    with tf.Session() as sess:
        
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        print (vgg_path)
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        image_input, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        
        output_layer = layers(layer3_out, layer4_out, layer7_out, num_classes)
        
        learning_rate = tf.placeholder(dtype = tf.float32)
        correct_label = tf.placeholder(dtype = tf.float32, shape = (None, None, None, num_classes)) 
        
        reshaped_logits, train_op, cross_entropy_loss = optimize(output_layer, 
                                                                 correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function
        sess.run(tf.global_variables_initializer())
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, 
                 image_input, correct_label, keep_prob, learning_rate )
        
        # TODO: Save inference data using helper.save_inference_samples
        
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape,reshaped_logits, keep_prob, image_input)
        
        # Save model weights to disk
        saver = tf.train.Saver()
        saver.save(sess, 'vgg_segment-model')
        print("Model saved in disk ")


        # OPTIONAL: Apply the trained model to a video

In [9]:
if __name__ == '__main__':
    
    run()

./data/vgg
('=== image_input ', TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(3)]))
('=== image_input ', <bound method Tensor.get_shape of <tf.Tensor 'image_input:0' shape=(?, ?, ?, 3) dtype=float32>>)
('=== keep_prob ', <bound method Tensor.get_shape of <tf.Tensor 'keep_prob:0' shape=<unknown> dtype=float32>>)
('=== layer3_out ', <bound method Tensor.get_shape of <tf.Tensor 'layer3_out:0' shape=(?, ?, ?, 256) dtype=float32>>)
('=== layer4_out ', <bound method Tensor.get_shape of <tf.Tensor 'layer4_out:0' shape=(?, ?, ?, 512) dtype=float32>>)
('=== layer7_out ', <bound method Tensor.get_shape of <tf.Tensor 'layer7_out:0' shape=(?, ?, ?, 4096) dtype=float32>>)
('Layers layer7 ', <bound method Tensor.get_shape of <tf.Tensor 'conv2d/BiasAdd:0' shape=(?, ?, ?, 2) dtype=float32>>)
('Layers layer4 ', <bound method Tensor.get_shape of <tf.Tensor 'conv2d_2/BiasAdd:0' shape=(?, ?, ?, 2) dtype=float32>>)
('Layers layer3 ', <bound method Tensor.get_shape of <tf.Tensor 

In [ ]:
import sys
import numpy as np
import glob
import os
import scipy
import matplotlib.image as img
import cv2
sys.version
sys.version_info

In [ ]:
background_color = np.array([255, 0, 0])
background_color.shape

In [ ]:
image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))

/home/tebd/Ashis_Samal/CarND-Semantic-Segmentation/data/data_road/training/image_2/
mpl.

In [ ]:
gt_bg= np.random.rand(160,576)
gt_bg.shape

In [ ]:
bool_arr = np.array([[True, False],[False, True]])

print (bool_arr)
bool_arr

In [ ]:
inv_bool_arr = np.invert(bool_arr)
inv_bool_arr

In [ ]:
bool_arr.shape

In [ ]:
bool_arr = np.reshape(bool_arr,(2,2,1))

In [ ]:
 np.concatenate((bool_arr,np.invert(bool_arr)), axis=2)

In [ ]:
image_shape = (576, 160)
image = cv2.resize(img.imread('/home/tebd/Ashis_Samal/CarND-Semantic-Segmentation/data/data_road/training/image_2/um_000000.png'), image_shape , interpolation = cv2.INTER_AREA)
#image.shape
gt_image = cv2.resize(img.imread('/home/tebd/Ashis_Samal/CarND-Semantic-Segmentation/data/data_road/training/gt_image_2/um_lane_000000.png'), image_shape , interpolation = cv2.INTER_AREA)
print ('===== gt_image ', gt_image.shape)

In [ ]:
gt_bg = np.all(gt_image == background_color, axis=2)
gt_bg.shape

In [ ]:
gt_bg

In [ ]:
gt_bg = gt_bg.reshape((160, 576),1)

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])

In [ ]:
a.shape

In [ ]:
np.concatenate((a, b), axis=0)

In [ ]:
#a.reshape(a.shape,1)
np.reshape(a, 6) 

In [ ]:
gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)